In [18]:
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from fastoad.io import VariableIO

COLS = plotly.colors.DEFAULT_PLOTLY_COLORS

def drone_geometry_plot(
    drone_file_path: str, name=None, fig=None, file_formatter=None
) -> go.FigureWidget:
    """
    Returns a figure plot of the top view of the wing.
    Different designs can be superposed by providing an existing fig.
    Each design can be provided a name.

    :param drone_file_path: path of data file
    :param name: name to give to the trace added to the figure
    :param fig: existing figure to which add the plot
    :param file_formatter: the formatter that defines the format of data file. If not provided, default format will
                           be assumed.
    :return: wing plot figure
    """
    
    variables = VariableIO(drone_file_path, file_formatter).read()
    if fig is None:
        fig = go.Figure()
    k = len(fig.data)
    
    N_arms = variables["data:structure:arms:number"].value[0]
    N_pro_arm = variables["data:structure:arms:prop_per_arm"].value[0]
    arm_length = variables["data:structure:arms:length"].value[0]
    arm_diameter = variables["data:structure:arms:diameter:outer"].value[0]
    D_pro = variables["data:propeller:geometry:diameter"].value[0]
    
    R = 0.0  # Frame radius #TODO: get frame radius

    x_arms = []
    y_arms = []
    prop_shapes = []
    
    for i in range(int(N_arms)):
        sep_angle = - i * 2 * np.pi / N_arms
        x_arm = np.array(
            [- arm_diameter / 2 * np.sin(sep_angle), - arm_diameter / 2 * np.sin(sep_angle) + arm_length * np.cos(sep_angle), arm_length * np.cos(sep_angle) + arm_diameter / 2 * np.sin(sep_angle), arm_diameter / 2 * np.sin(sep_angle)]
        )
        y_arm = np.array(
            [arm_diameter / 2 * np.cos(sep_angle), arm_diameter / 2 * np.cos(sep_angle) + arm_length * np.sin(sep_angle), arm_length * np.sin(sep_angle) - arm_diameter / 2 * np.cos(sep_angle), - arm_diameter / 2 * np.cos(sep_angle)]
        )
        
        x_arm = x_arm + R * np.cos(sep_angle)  # frame radius
        y_arm = y_arm + R * np.sin(sep_angle)  # frame radius
        
        x_arms = np.concatenate((x_arms, x_arm))
        y_arms = np.concatenate((y_arms, y_arm))
        
        for j in range(int(N_pro_arm)):  # TODO: distinguish the two propellers if push/pull configuration
            fig.add_shape(
                dict(type="circle", line=dict(color='black',width=3), fillcolor=COLS[k],
                          x0=arm_length * np.cos(sep_angle) - D_pro / 2 + R * np.cos(sep_angle), 
                          y0=arm_length * np.sin(sep_angle) - D_pro / 2 + R * np.sin(sep_angle), 
                          x1=arm_length * np.cos(sep_angle) + D_pro / 2 + R * np.cos(sep_angle), 
                          y1=arm_length * np.sin(sep_angle) + D_pro / 2 + R * np.sin(sep_angle),
                    )
                )

    # ARMS
    x_arms = np.concatenate((x_arms,[x_arms[0]]))
    y_arms = np.concatenate((y_arms,[y_arms[0]]))
    scatter = go.Scatter(x=y_arms, y=x_arms, mode="lines", line_color = COLS[k], name=name)
    fig.add_trace(scatter)
    
        
    fig = go.FigureWidget(fig)
    fig.update_shapes(opacity=0.2, xref="x", yref="y")
    fig.update_layout(
        title_text="Drone Geometry", title_x=0.5, xaxis_title="y", yaxis_title="x",
    )
    fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1))

    return fig

In [19]:
import os.path as pth
from fastoad.io import VariableIO

DATA_FOLDER_PATH = '../data'
WORK_FOLDER_PATH = '../workdir'

MDA_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mda.xml')
MDO_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mdo.xml')

fig = drone_geometry_plot(MDA_OUTPUT_FILE, name='Drone MDA')
fig = drone_geometry_plot(MDO_OUTPUT_FILE, name='Drone MDO', fig=fig)
fig.show()

In [12]:
str = 'rgba(' + COLS[0] + ',0.5)'
print(str)
#plotly.colors.convert_colorscale_to_rgb(COLS[0])

rgba(rgb(31, 119, 180),0.5)
